In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re

referenceAvailable = True

# %matplotlib widget

# DP Simulation of topology with slack, line and VSI

## DP simulation

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

DURATION=8.0
TIMESTEP=3e-3

DP_Slack_PiLine_VSI_Ramp_with_PF_Init --timestep=${TIMESTEP} --duration=${DURATION}

In [ ]:
modelName = 'DP_Slack_PiLine_VSI_Ramp_with_PF_Init_DP'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)
ts_dpsim_shifted = ts.frequency_shift_list(ts_dpsim, 50)

## DP and EMT simulation for reference

In [ ]:
if referenceAvailable:
    modelName = 'DP_Slack_PiLine_VSI_Ramp_with_PF_Init_DP'
    path = 'logs/' + modelName + '/'
    dpsim_result_file = path + modelName + '.csv'

    ts_dpsim_ref = read_timeseries_csv(dpsim_result_file)

In [ ]:
if referenceAvailable:
    modelName = 'EMT_Slack_PiLine_VSI_Ramp_with_PF_Init_EMT'
    path = 'logs/' + modelName + '/'
    dpsim_result_file = path + modelName + '.csv'

    ts_dpsim_EMT = read_timeseries_csv(dpsim_result_file)

## Plots

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
        if ts_name in ['f_src']:
                plt.plot(ts_obj.time, ts_obj.values, label=ts_name, color='C0')

plt.xlim(4.8,8)
plt.xlabel('Zeit [s]')
plt.ylabel('Frequenz der Spannungsquelle [Hz]')
plt.gcf().legend(loc='upper center', ncol=2)
plt.show()

In [ ]:
plt.figure(figsize=(8,4.2))
if referenceAvailable:
    for ts_name, ts_obj  in ts_dpsim_EMT.items():
            if ts_name in ['v2_0']:
                    plt.plot(ts_obj.time, np.sqrt(3/2)*ts_obj.values, label='u2 abs (EMT, 50 µs ref)')
    for ts_name, ts_obj  in ts_dpsim_ref.items():
            if ts_name in ['v2']:
                    plt.plot(ts_obj.time, ts_obj.abs().values, label='u2 abs (DP, 50 µs ref)', color='C1')
for ts_name, ts_obj  in ts_dpsim.items():
        if ts_name in ['v2']:
                plt.plot(ts_obj.time, ts_obj.abs().values, label='u2 abs (DP, 1 ms)', color='C1', linestyle='--')

plt.xlim(4.8,8)
plt.ylim(20014,20030)
plt.xlabel('Zeit [s]')
plt.ylabel('Betrag der Lastspannung u2 [V]')
plt.gcf().legend(loc='upper center', ncol=3)
plt.grid('y')
plt.show()

### Assertion

In [ ]:
if referenceAvailable:
    compare_errors_abs = []
    for name in [('v2_0', 'v2_shift')]:
        compare_errors_abs.append(np.absolute(np.sqrt(3/2)*ts_dpsim_EMT[name[0]].values[80000:] - ts_dpsim_shifted[name[1]].interpolate(50e-6).values[80000:160002]).max())
        print(name[0]+' vs. '+name[1] + ' (abs): ' + str(compare_errors_abs[-1]))
    assert np.max(compare_errors_abs) < 3000